In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from pathlib import Path

repo_dir = '/home/labs/amit/noamsh/repos/MM_2023'
sys.path.append(repo_dir)

In [6]:
import pandas as pd
import pyreadr

In [7]:
shuangyi_MM_dir = '/home/labs/amit/shuangyi/Project_MM_2023/'
spid_label_path = shuangyi_MM_dir + "/ann_SPID_20231110_CD45.Rds"
mars_label_path = shuangyi_MM_dir + "/ann_MARS_20231110_CD45.Rds"

mars_result = pyreadr.read_r(mars_label_path)
spid_result = pyreadr.read_r(spid_label_path)

df_mars_labels = mars_result[None] 
df_spid_labels = spid_result[None] 

label_col = "Populations"
annot_labels = pd.concat([df_mars_labels[[label_col, "cID"]], df_spid_labels[[label_col, "cID"]]])

In [8]:
cells_dir = "/home/labs/amit/noamsh/data/mm_2023/cells"
spid_PC_cells_path = cells_dir + "/spid_PC_cells.csv"
mars_PC_cells_path = cells_dir + "/mars_PC_cells.csv"

mars_PC_cID = pd.read_csv(mars_PC_cells_path)['Unnamed: 0']
spid_PC_cID = pd.read_csv(spid_PC_cells_path)['Unnamed: 0']

# mars_PC_cID
all_PC = pd.concat([mars_PC_cID, spid_PC_cID]).reset_index(drop=True)

spid_CD45_cells_path = cells_dir + "/spid_CD45_cells.csv"
mars_CD45_cells_path = cells_dir + "/mars_CD45_cells.csv"

mars_CD45_cID = pd.read_csv(mars_CD45_cells_path)['Unnamed: 0']
spid_CD45_cID = pd.read_csv(spid_CD45_cells_path)['Unnamed: 0']

all_CD45 = pd.concat([mars_CD45_cID, spid_CD45_cID]).reset_index(drop=True)

super_pop = "super_Population"
super_pop_df = pd.concat([pd.DataFrame(data={"cID": all_PC, super_pop: "PC"}),
                           pd.DataFrame(data={"cID": all_CD45, super_pop: "CD45"})]).reset_index(drop=True)

In [11]:
print(f"""
annot - {len(annot_labels["cID"])},
CD45 cells - {len(all_CD45)}, 
PC cells - {len(all_PC)},
annot-CD45 intersection - {len(set(annot_labels["cID"]).intersection(all_CD45.values))},
annot-PC intersection - {len(set(annot_labels["cID"]).intersection(all_PC.values))})
""")


annot - 182924,
CD45 cells - 183507, 
PC cells - 170044,
annot-CD45 intersection - 182924,
annot-PC intersection - 0)



In [12]:
# from omegaconf import OmegaConf

# config_path = Path(repo_dir, 'config.yaml')
# conf = OmegaConf.load(config_path)

# adata_path = Path(conf.outputs.output_dir, conf.outputs.processed_adata_file_name)
# adata_pp = ad.read_h5ad(adata_path)

# f"""
# pp_adata - {len(adata_pp)}
# pp_adata-CD45 intersection - {len(set(adata_pp.obs_names).intersection(all_CD45.values))} 
# pp_adata-PC intersection - {len(set(adata_pp.obs_names).intersection(all_PC.values))}
# pp_adata-annot intersection - {len(set(adata_pp.obs_names).intersection(all_labels["cID"]))}

# pp_adata-no_label - {len(adata_pp) - len(set(adata_pp.obs_names).intersection(pd.concat([all_CD45, all_PC]).values))}
# """)

In [14]:
all_labels = super_pop_df.merge(annot_labels, on="cID", how="left")
all_labels

,cID,super_Population,Populations
0,W555665,PC,NaN
1,W555666,PC,NaN
2,W555667,PC,NaN
3,W555668,PC,NaN
4,W555669,PC,NaN
...,...,...,...
353546,W4560721,CD45,NK
353547,W4560722,CD45,Mo
353548,W4560724,CD45,NK
353549,W4560726,CD45,Mo_Pro


In [15]:
all_annotation_path = Path(cells_dir, "cells_snnotation_20231110.csv")
all_labels.to_csv(all_annotation_path, index=False)